In [24]:
import timeit

In [25]:
import sys
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
np.random.seed(42)

NUM_FEATS = 5

class Net(object):
    '''
    '''

    def __init__(self, num_layers, num_units):
        
        
        self.num_layers = num_layers
        self.num_units = num_units
        

        self.biases = []
        self.weights = []
        for i in range(num_layers):

            if i==0:
                # Input layer
                self.weights.append(np.random.uniform(0, 1, size=(NUM_FEATS, self.num_units)))

            else:
                # Hidden layer
                self.weights.append(np.random.uniform(0, 1, size=(self.num_units, self.num_units)))


            self.biases.append(np.random.uniform(0, 1, size=(1, self.num_units)))


        # Output layer
        self.biases.append(np.random.uniform(0, 1, size=(1, 1)))
        self.weights.append(np.random.uniform(0, 1, size=(self.num_units, 1)))

    def __call__(self, X):
       
        a=X
        weights = self.weights
        biases = self.biases
        self.a_state = []
        self.h_state = []
        self.a_state.append(a)
        for i in range(len(self.weights)):
            h = np.dot(a, weights[i]) + biases[i]
            if i<len(self.weights)-1:
                self.h_state.append(h)
                a = np.maximum(h, 0)
                self.a_state.append(a)
            else:
                #self.h_state.append(h)
                a=h
                self.a_state.append(a)
                
        y_hat=a
        return y_hat
        raise NotImplementedError

    def backward(self, X, y, lamda):
        

        del_W = []
        del_b = []
        weights = self.weights
        biases = self.biases
        
        y = np.reshape(y, (y.shape[0], 1))
        n = y.shape[0]
        
        
        a_state=self.a_state
        h_state=self.h_state
        
        del1 = (1/n) * (a_state[-1] - y)
        
        delw = np.dot(a_state[-2].T, del1) + lamda * (weights[-1])
        
        delb = np.sum(del1, axis=0) + lamda * (biases[-1])
        
        del_W.append(delw)
        del_b.append(delb)

        
        for i in range(self.num_layers):
            del1=np.dot(del1,weights[-(i+1)].T)
            delw=np.dot(a_state[-(i+3)].T,(del1*(h_state[-(i+1)]>0)))+lamda*(weights[-(i+2)])
            delb=np.sum(del1*(h_state[-(i+1)]>0),axis=0) + lamda*(biases[-(i+2)])
            del_W.insert(0,delw)
            del_b.insert(0,delb)
        

        return del_W, del_b
        raise NotImplementedError



In [26]:

class Optimizer(object):
    '''
    '''

    def __init__(self, learning_rate):
        
        self.learning_rate = learning_rate
        

    def step(self, weights, biases, delta_weights, delta_biases):
        
        for i in range(len(weights)):
            weights[i] =weights[i] -self.learning_rate * delta_weights[i]
            biases[i] =biases[i]- self.learning_rate * delta_biases[i]

        return weights, biases
        raise NotImplementedError



In [27]:

def loss_mse(y, y_hat):
    
    m = y.shape[0]
    y = np.reshape(y, (y.shape[0], 1))
    mse = np.sum(np.square(y - y_hat))
    mse=(1/m)*mse
    return mse

    raise NotImplementedError

def loss_regularization(weights, biases):

    l2_reg = 0

    for i in range(len(weights)):
        l2_reg += np.sum(np.square(weights[i])) + np.sum(np.square(biases[i]))
    
    return l2_reg
    raise NotImplementedError

def loss_fn(y, y_hat, weights, biases, lamda):
    mse_ls=loss_mse(y, y_hat)
    reg_ls=loss_regularization(weights, biases)
    loss=mse_ls+lamda*reg_ls
    return loss
    raise NotImplementedError

def rmse(y, y_hat):
    
    rsme = np.sqrt(loss_mse(y, y_hat))
    return rsme
    raise NotImplementedError




In [28]:
def train(
    net, optimizer, lamda, batch_size, max_epochs,
    train_input, train_target,test_input,test_target):
   

    m = train_input.shape[0]

    for e in range(max_epochs):
        epoch_loss = 0.
        for i in range(0, m, batch_size):
            batch_input = train_input[i:i+batch_size]
            batch_target = train_target[i:i+batch_size]
            
            pred = net(batch_input)
            #print(pred)
            # Compute gradients of loss w.r.t. weights and biases
            dW, db = net.backward(batch_input, batch_target, lamda)
            #print(dW)

            # Get updated weights based on current weights and gradients
            weights_updated, biases_updated = optimizer.step(net.weights, net.biases, dW, db)

            # Update model's weights and biases
            net.weights = weights_updated
            net.biases = biases_updated

            # Compute loss for the batch
            batch_loss = loss_fn(batch_target, pred, net.weights, net.biases, lamda)
            epoch_loss += batch_loss
#             train_pred=net(train_input)
#             train_rmse=rmse(train_target,train_pred)
#             print('RMSE on train data: {:.5f}'.format(train_rmse))

            #print(e, i, rmse(batch_target, pred), batch_loss)
            
        #print(e, epoch_loss)
        
#         dev_pred=net(dev_input)
#         dev_rmse = rmse(dev_target,dev_pred)
#         print('RMSE on dev data: {:.5f}'.format(dev_rmse))
        
#     dev_pred=net(dev_input)
#     #print(dev_pred)
#     dev_rmse = rmse(dev_target,dev_pred)
    test_pred=net(test_input)
    test_rmse=rmse(test_target,test_pred)
    print('RMSE on test data: {:.5f}'.format(test_rmse))
    #print('RMSE on dev data: {:.5f}'.format(dev_rmse))
    #print(test_pred)
        



def read_data():
    '''
    Read the train, dev, and test datasets
    '''
    data=pd.read_csv('NSE-TATAGLOBAL11.csv')
    X=data.drop(['Date','St Date','Close','Turnover (Lacs)','Total Trade Quantity'],axis=1)
    Y=data['Close']
    train_input_df,test_input_df,train_target_df,test_target_df=train_test_split(X,Y,test_size=0.1,random_state=44)
    #type(train_input)
    train_input=np.array(train_input_df)
    train_target=np.array(train_target_df)
    test_input=np.array(test_input_df)
    test_target=np.array(test_target_df)
#     print(train_input.shape)
    return train_input, train_target,test_input,test_target


def main():

    # These parameters should be fixed for Part 1
    max_epochs = 100
    batch_size = 1


    learning_rate = 2e-10
    num_layers = 2
    num_units = 7
    lamda = 0.1 # Regularization Parameter

    train_input, train_target,test_input,test_target = read_data()
    net = Net(num_layers, num_units)
    optimizer = Optimizer(learning_rate)
    train(
        net, optimizer, lamda, batch_size, max_epochs,
        train_input, train_target,test_input,test_target)
    #get_test_data_predictions(net, test_input)

start=timeit.default_timer()
if __name__ == '__main__':
    main()
stop=timeit.default_timer()
print('Time:',stop-start)

RMSE on test data: 9.17496
Time: 34.25709589999951
